In [ ]:
using Distributed

In [ ]:
include("./base_utils.jl")

In [ ]:
include("./dmrg.jl");

In [ ]:
include("./time_evo.jl");

In [ ]:
include("./RUC.jl");

In [ ]:
include("sampling.jl");

# Test DMRG

In [ ]:
cols=3
rows=3
config=zeros(Int,rows,cols)
k=0
for j in 1:cols
    for i in 1:rows
        k+=1
        config[i,j]=k
    end
end
config

In [ ]:
d=2
chi=10
chimax=64
sweeps=10
J1=1.
J2=0.5
M=MPO()
Initialize!("J1-J2_ITensors",M,J1,J2,cols,rows,config);

In [ ]:
ψ=MPS()
Initialize!(ψ,d,chi,cols*rows)
two_sites_dmrg!(ψ,M,sweeps,chimax)

# Test RUC

In [ ]:
include("./RUC.jl")
include("./time_evo.jl")

In [ ]:
ψ=MPS()
N=10
d=1
Initialize!("product_state",ψ,repeat(["up","down"],Int(N/2)))

In [ ]:
Ulist=RUC_layer(ψ.N);
print(size(Ulist))

In [ ]:
ψt=copy(ψ)
sweeps=1
trotter_evolution!(ψt,Ulist,sweeps)

In [ ]:
sweeps=1
nruns=50
evosteps=50
ψ=MPS()
N=10
Initialize!("product_state",ψ,repeat(["up","down"],Int(N/2)))
prange=0:0.05:1
Observables=zeros(length(prange),nruns,evosteps+1,N)
for (ip,p) in enumerate(prange)
    println("p= ",p," and run: ")
    
    for run in 1:nruns
        print(run," ")
        #print("time step: ")
        ψt=copy(ψ)
        Observables[ip,run,1,:]=calc_entropy(ψt)
        for i in 1:evosteps
            #print(i," ")
            Ulist=RUC_layer(ψt.N);
            trotter_evolution!(ψt,Ulist,sweeps)
            RUC_measure!(ψt,p)
            Observables[ip,run,i+1,:]=calc_entropy(ψt)        
            #=if mod(i,interval)==0
                print(step," ")
                step+=1
                cc[step,:]=sample_configs(ψt,1)
                ent[step,:]=calc_entropy(ψt)
                #projection
                P=MPO()
                Initialize!("proj",P,cc[step,:],Array(1:ψt.N),ψt.N)
                ψt=P*ψt
                right_normalize!(ψt)
            end
            =#

        end
        #println()
    end
    println()
end

In [ ]:
using Plots

In [ ]:
plot([mean(Observables[21,:,i,Int(N/2)+1]) for i in 1:evosteps+1])

In [ ]:

println([mean(Observables[[21,:,i,Int(N/2)+1]) for i in 1:evosteps+1])